In [1]:
import numpy as np
from os import listdir
from os.path import isfile, join

mypath = './example_Kr_docs/'
onlyfiles = [f for f in listdir(mypath) if isfile(join(mypath, f))]
onlyfiles.sort()

total_docs = []
for file in onlyfiles:
    file_path = mypath+file
    with open(file_path, 'r', encoding='utf8') as f:
        content = f.read()
    total_docs.append(content)

In [2]:
len(total_docs)

13

In [3]:
total_docs[0]

'정기석 코로나19(COVID-19) 특별대응단장 겸 국가 감염병 위기대응 자문위 위원장이 개량백신의 부작용은 이전 백신의 10분의 1 수준이라며 다시 한번 동절기 추가접종을 당부했다.\n\n5일 정 위원장은 "백신 접종이 그 어느 때보다 중요한 시기"라며 "지금 국가에서 제공하고 있는 코로나19 개량백신은 그 전에 우리가 4차까지 맞았던 백신과 내용이 다르고 목표도 다르고, 그래서 백신 접종 제고를 위해 범정부적·전방위적으로 노력해야 한다"고 밝혔다.\n\n또 "지난 11월 21일부터 이달 18일까지 집중접종기간 안에 60세 이상 국민의 50% 이상, 감염취약시설에 거주자 및 종사자의 60% 이상 백신 접종이 이뤄져야 가장 위험한 그분들의 생명을 보호할 수 있다"고 강조했다.\n\n이어 "정부는 의료계와 협력해 의료 현장에서 접종 권고가 더 적극적으로 이뤄지도록 소통과 홍보를 하고, 지자체(지방자치단체)는 접종 현황을 매일 점검해 지역 특성을 고려한 다양한 접종 편의를 제공해야 한다"고 말했다.\n\n정 위원장은 우리 국민이 코로나19 백신 접종을 주저하는 이유로 약한 감염 예방 효과, 부작용 우려를 꼽았다.\n\n정 위원장은 "예방접종 효과에 대한 불신과 이상반응에 대한 우려 등을 해소하기 위해 객관적이고 정확한 정보와 계층별 맞춤 정보를 다양하게 제공해야 한다"며 "현재 나오고 있는 개량백신에 대한 부작용률은 그 전 백신에 비해 10분의 1밖에 되지 않는 매우 미미한 부작용률을 보이고 있다"고 말했다.\n\n또 "그 전에는 (백신을) 맞아도 (코로나19에) 걸렸고, 저도 맞고 걸렸다"며 "구형 백신을 맞았을 때 오미크론을 막아주는 능력이 매우 떨어졌다"고 말했다.\n\n이어 "그러나 지금 개량백신은 BA.5를 막아준다"며 "BA.5는 공식적으로 국내 발생의 80%를 차지하고, BA.5 계열까지 합치면 85%를 차지한다"고 설명했다.'

In [4]:
# 키위 형태소 분석기를 사용한 전처리
from kiwipiepy import Kiwi
from kiwipiepy.utils import Stopwords
import re
kiwi = Kiwi()
stopwords_dict = Stopwords()

In [5]:
# 전처리를 위한 사용자 정의 함수
def Kr_preprocessing(text):
    text = text.strip()
    text = re.sub(r'[^\d\s\w]',' ',text)
    kiwi_tokens = kiwi.tokenize(text, stopwords=stopwords_dict)
    noun_words=[]
    for token in kiwi_tokens:
        if 'NN' in token.tag and len(token.form)>1:
            noun_words.append(token.form)
    return ' '.join(noun_words)

In [6]:
filtered_docs = [Kr_preprocessing(doc) for doc in total_docs]

In [7]:
len(filtered_docs)

13

In [8]:
filtered_docs[0]

'정기 코로나 특별 대응 단장 국가 감염병 위기 대응 자문위 위원장 개량 백신 부작용 이전 백신 수준 동절기 추가 접종 당부 위원장 백신 접종 중요 시기 국가 제공 코로나 개량 백신 백신 내용 목표 백신 접종 제고 정부 전방위 노력 이달 집중 접종 기간 이상 국민 이상 감염 취약 시설 거주자 종사자 이상 백신 접종 위험 생명 보호 강조 정부 의료 협력 의료 현장 접종 권고 적극 소통 홍보 지자체 지방 자치 단체 접종 현황 점검 특성 고려 다양 접종 편의 제공 위원장 국민 코로나 백신 접종 주저 이유 감염 예방 효과 부작용 우려 위원장 예방 접종 효과 불신 이상 반응 우려 해소 객관 정확 정보 계층 맞춤 정보 다양 제공 개량 백신 부작용 백신 부작용 백신 코로나 구형 백신 미크론 능력 개량 백신 공식 국내 발생 차지 계열 차지 설명'

### TF-IDF 기반 DTM에 KMeans 적용하기

In [9]:
from sklearn.feature_extraction.text import TfidfVectorizer # tf-idf based DTM
tfidf_vectorizer = TfidfVectorizer()
DTM_tfidf = tfidf_vectorizer.fit_transform(filtered_docs)
DTM_TFIDF = np.array(DTM_tfidf.todense())

In [37]:
DTM_TFIDF.shape

(13, 642)

In [38]:
from sklearn.cluster import KMeans

In [41]:
kmeans = KMeans(n_clusters=4)
clusters_TFIDF = kmeans.fit_predict(DTM_TFIDF)

In [42]:
clusters_TFIDF

array([1, 1, 1, 1, 0, 0, 0, 2, 2, 2, 3, 3, 3])